# Quick projections of multichannel 3D images (```.nd2``` Version) 

## Input

This recipe expects an input folder containing 3D & multichannel ```.nd2``` files.

**NOTE:** single channel images are untested at the moment.

## Output

For each channel of each ```.nd2``` image in the input folder, a ```.png``` image showning z-,y- and x-projections will be created in the output folder. The name of output files will be ```{input file name (without ending)}_{channel name}_projected.png```

In [ ]:
# imports / definitions
import warnings
from pathlib import Path
from skimage.io import imsave
from utils.projection import load_multichannel_nd2, get_orthogonal_projections_8bit

In [ ]:
# path containing files to visualize
in_path = '/home/stumberger/ep2024/example/raw/'

# default: put results in subdirectory called 'projections'
out_subdirectory = 'projections'

# Intensity range for levelling
# Default: 'auto' -> level based on quantiles 
intensity_range = 'auto'
auto_range_quantiles = (0.02, 0.9995)

# Alternatively, specify range directly
# intensity_range = (500, 1500)

# Type of projection, can be min/max/mean
projection_type='max'

In [ ]:
out_path = Path(in_path) / out_subdirectory

# get all nd2 files in in_path
in_files = sorted(list(Path(in_path).glob('*.nd2')))

# show for verification
in_files

In [ ]:
# make folder for results if it does not exist
if not out_path.exists():
    out_path.mkdir(parents=True)

for in_file in in_files:

    # read images, channel names, pixel sizes
    try:
        imgs, channel_names, pixel_size = load_multichannel_nd2(in_file)
    except:
        print(f'error loading file {in_file}, skipping')
        continue


    # do projections and save
    for img, channel_name in zip(imgs, channel_names):

        projections_8bit = get_orthogonal_projections_8bit(img, pixel_size,
                projection_type=projection_type, intensity_range=intensity_range, auto_range_quantiles=auto_range_quantiles)

        # make filepath for output
        outfile = out_path / (in_file.stem + f'_{channel_name}_projected.png')
        
        # catch low contrast warning
        with warnings.catch_warnings():
            warnings.simplefilter('ignore', UserWarning)
            
            imsave(str(outfile), projections_8bit)
    
    print(f'saved projections of {str(in_file)}.')